## Sample Data Generation

This script generates artificial log files by simulating user activities 

The data is used to train and test different models (like an hidden markov model, recurrent neural networks and generative adversial networks to detect outliers / fraud in user transactional behavior. For example the employee usage of a CRM system to detect data theft or customer behavior in a online shop to sport fraudulent orders.

Simulation is done via Markov Chain Monte Carlo Simulation.

We define a set of possible activities like (start, end, action_1, ..., action_n) and a set of users. Each user belong to one of n classes (eg. normal behavior and fraudulent behavior). For each class we have a 
transition matrix giving the probabilities that a user perform a specific action given his previous action and his state.


In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
# Configuration

# first two actions need to be start and end
actions = ['start', 'end', 'action_1', 'action_2', 'action_3']

n_users = 100

n_states = 2

# Probability that a user is one of these states
p_states = [0.98, 0.02]

n_activities_per_user_per_day = 100

n_days = 1

transitions = [
    # Normal behavior
    np.array([
        [0.00, 0.00, 1/3,  1/3,  1/3],
        [1.00, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.01, 0.09, 0.45, 0.45],
        [0.00, 0.88, 0.01, 0.10, 0.01],
        [0.00, 0.68, 0.01, 0.30, 0.01],
    ]),
    # Fraudulent Behavior
    np.array([
        [0.00, 0.00, 0.8,  0.1,  0.1],
        [1.00, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.2, 0.70, 0.05, 0.05],
        [0.00, 0.88, 0.01, 0.10, 0.01],
        [0.00, 0.68, 0.01, 0.30, 0.01],
    ])
]

assert len(p_states)==n_states, 'Inconsitent number of states and state probs'
assert np.sum(p_states)==1, 'State probs doesnt sum up to one'
assert len(transitions)==n_states, 'Inconsitent number of transition matrixes and state'
for i in range(n_states):
    assert np.allclose(transitions[i].sum(), len(actions))
    assert np.allclose(transitions[i].cumsum(axis=1)[:,-1],1)

In [4]:
# Simulation
def simulate_markov_chain(transitions, state_id, actions):
    '''
    Simulate a user session using the corresponding transition matrix
    given the current user state
    
    Parameter:
    
        transitions : list of transition matrixes
        state_id: int of the current user state
        actions: list of all available actions
        
    Returns:
    
        list of artificial activities in one session
    '''
    activities = [actions[0]]
    while activities[-1] != 'end':
        prev_action = actions.index(activities[-1])
        transition_probs = transitions[state_id][prev_action]
        next_action = np.random.choice(actions, size=1, p=transition_probs)[0]
        activities.append(next_action)
    return activities


np.random.seed(42)
log_data = []
for user in range(n_users):
    for day in range(n_days):
        current_state = np.random.choice(np.arange(0, n_states), size=1, p=p_states)[0]
        for i in range(n_activities_per_user_per_day):
            activities = simulate_markov_chain(transitions, current_state, actions)
            log_data.append((user, day, i, activities, current_state))

In [5]:
logfile = pd.DataFrame(log_data, columns=('UserID', 'Day', 'UserSessionId', 'SessionActivity', 'State'))

In [6]:
%%bash 
mkdir Data

In [7]:
logfile.to_pickle('./Data/logfile_v1.pkl')

In [13]:
id2actions = np.array(['<pad>']+actions)
np.save('./Data/id2action', id2actions)

In [14]:
%%bash

cd Data
ls -lisa

total 4208
8604106216    0 drwxr-xr-x  4 matthiasgroncki  staff      128 Nov  8 16:11 .
8604100559    0 drwxr-xr-x  8 matthiasgroncki  staff      256 Nov  8 16:11 ..
8604106313    8 -rw-r--r--  1 matthiasgroncki  staff      320 Nov  8 16:11 id2action.npy
8604106218 4200 -rw-r--r--  1 matthiasgroncki  staff  2147215 Nov  8 16:10 logfile_v1.pkl
